<a href="https://colab.research.google.com/github/lautarogerlero/Data-Engineering-Practice/blob/main/Admission_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install PySpark and create a session


In [1]:
#install pyspark
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=aadc468f176e973d37d2e9d089231a8bad97b39ca5867005c57fe0d9b5b7fe7d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
#create a sparksession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark").getOrCreate()

Clone & explore the dataset

In [3]:
#clone the dataset
! git clone https://github.com/education454/admission_dataset

Cloning into 'admission_dataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
Receiving objects: 100% (3/3), 5.60 KiB | 5.60 MiB/s, done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0


In [4]:
#check the presence of dataset
! ls admission_dataset

Admission_Predict_Ver1.1.csv


In [5]:
#create a spark dataframe
df = spark.read.csv("/content/admission_dataset/Admission_Predict_Ver1.1.csv", header=True, inferSchema=True)

In [6]:
#display dataframe
df.show()

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|        7|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|        8|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|        9

In [7]:
#get the no.of rows & columns
print((df.count(), len(df.columns)))

(500, 9)


In [8]:
#print schema
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [9]:
#get the summary statistics
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

Data cleaning

In [10]:
#drop the unnecessary column
df = df.drop("Serial No")

In [11]:
#display the dataframe
df.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|      302|        102|                1|2.0|1.5| 8.0|       0|            0.5|
|      323|        108|                3

In [12]:
#check for null values
for i in df.columns:
  print(i+":", df[df[i].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


Correlation Analysis & Feature Selection

In [13]:
# correlation analysis
for col in df.columns:
  print("Correlation to chance of admit col for {} is {}".format(col, df.stat.corr("Chance of Admit", col)))

Correlation to chance of admit col for GRE Score is 0.8103506354632598
Correlation to chance of admit col for TOEFL Score is 0.7922276143050823
Correlation to chance of admit col for University Rating is 0.6901323687886892
Correlation to chance of admit col for SOP is 0.6841365241316723
Correlation to chance of admit col for LOR is 0.6453645135280112
Correlation to chance of admit col for CGPA is 0.882412574904574
Correlation to chance of admit col for Research is 0.5458710294711379
Correlation to chance of admit col for Chance of Admit is 1.0


In [14]:
# feature selection
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["GRE Score", "TOEFL Score", "CGPA"], outputCol="features")

In [15]:
#display dataframe
output_data = assembler.transform(df)
output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

Building the Linear Regression Model

In [16]:
#import Linearregression and create final data
from pyspark.ml.regression import LinearRegression
final_data = output_data.select("features", "Chance of Admit")

In [17]:
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [18]:
#split the dataset into training and testing set
train, test = final_data.randomSplit([0.7, 0.3])

In [19]:
#build & train the model
models = LinearRegression(featuresCol="features", labelCol="Chance of Admit")
model = models.fit(train)

In [20]:
#get coefficients & intercept
print("coefficients: ", model.coefficients)
print("intercept: ", model.intercept)

coefficients:  [0.0027072084675359358,0.0029146580031848934,0.13958097200873618]
intercept:  -1.643123720952607


In [21]:
#get summary of the model
summary = model.summary

In [22]:
#print the rmse & r2 score
print("RMSE", summary.rootMeanSquaredError)
print("R2 Score", summary.r2)

RMSE 0.05999941005706698
R2 Score 0.8125942939055157


Evaluating & saving the Model

In [23]:
#transform on the test data
predictions = model.transform(test)

In [24]:
#display the predictions
predictions.show()

+------------------+---------------+-------------------+
|          features|Chance of Admit|         prediction|
+------------------+---------------+-------------------+
|[290.0,104.0,7.46]|           0.45|0.48636521814921485|
| [294.0,93.0,7.36]|           0.46|0.45117471678345167|
| [294.0,95.0,7.64]|           0.49|0.49608670495226725|
| [295.0,99.0,7.57]|           0.37| 0.5006818773919313|
| [296.0,95.0,7.54]|           0.44| 0.4875430246864654|
|  [296.0,97.0,7.8]|           0.49| 0.5296633934151065|
|[296.0,101.0,7.68]|            0.6|  0.524572308786798|
| [297.0,100.0,7.9]|           0.52| 0.5550726730930711|
| [298.0,92.0,7.88]|           0.51|  0.531670998094953|
|[298.0,100.0,7.95]|           0.58| 0.5647589301610436|
|[298.0,101.0,7.69]|           0.53| 0.5313825354419572|
|[299.0,100.0,7.88]|           0.68|  0.557695470587968|
|[299.0,100.0,7.89]|           0.59| 0.5590912803080554|
| [299.0,106.0,8.4]|           0.64| 0.6477655240516205|
|[300.0,102.0,7.87]|           

In [25]:
#evaluate the model
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Chance of Admit", metricName="r2")
print("r2 on the test data", evaluator.evaluate(predictions))

r2 on the test data 0.7875359497349875


In [26]:
#save the model
model.save("model")

In [27]:
#load the model
from pyspark.ml.regression import LinearRegressionModel
model = LinearRegressionModel.load("model")